In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Subset

def create_dataset(data):
    labels = data[..., 0]
    features = data[..., 1:]
    features = features.permute(0, 3, 1, 2).contiguous()
    dataset = TensorDataset(features, labels)
    return dataset


def train_val_test_split(dataset, val_indices, test_indices):
    all_indices = set(range(len(dataset)))
    train_indices = list(all_indices - set(val_indices) - set(test_indices))
    train_indices.sort()
    train_dataset = Subset(dataset, train_indices)
    val_dataset = Subset(dataset, val_indices)
    test_dataset = Subset(dataset, test_indices)
    return train_dataset, val_dataset, test_dataset


data = torch.load("../data_to_train/tensors.pt")
dataset = create_dataset(data)
val_indices = [20, 33, 52, 71, 90, 110, 132, 155, 177, 197, 219, 237, 250, 257, 22, 39, 58, 77, 96, 116, 138, 161, 183, 203, 225, 243]
test_indices = [21, 34, 53, 72, 91, 111, 133, 154, 178, 198, 220, 238, 251, 258, 23, 40, 59, 78, 97, 117, 139, 162, 184, 204, 226, 244]
train_dataset, val_dataset, test_dataset = train_val_test_split(dataset, val_indices, test_indices)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Train dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))
print("Test dataset size:", len(test_dataset))

Train dataset size: 210
Validation dataset size: 26
Test dataset size: 26


/tmp/ipykernel_140545/805229967.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("../data_to_train/tensors.pt")


In [21]:
X_train = data[..., 1:]
Y_train = data[..., 0]
print(X_train.shape)
print(Y_train.shape)

torch.Size([262, 26, 25, 6])
torch.Size([262, 26, 25])


In [22]:
X_train = torch.rand(1000, 6, 26, 25)
Y_train = torch.randint(0, 18 + 1, (1000, 26, 25)) * 5
print(X_train.shape)
print(Y_train.shape)

torch.Size([1000, 6, 26, 25])
torch.Size([1000, 26, 25])


In [23]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Set CUDA_LAUNCH_BLOCKING for better error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

class PixelClassifierCNN(nn.Module):
    def __init__(self, num_features=6, num_classes=18):
        super(PixelClassifierCNN, self).__init__()
        self.conv1 = nn.Conv2d(num_features, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc = nn.Conv2d(128, num_classes, kernel_size=1)  # 1x1 Conv for per-pixel classification
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.fc(x)
        return x  # Output shape: (batch, num_classes, 26, 25)

# Dane wejściowe
batch_size = 32
num_classes = 18
X_train = torch.rand(1000, 6, 26, 25)  # 1000 obrazków z 6 kanałami
Y_train = torch.randint(0, num_classes + 1, (1000, 26, 25)) * 5  # Label dla każdego piksela (0, 5, 10, ..., 90)
# Y_train = data[..., 0]
# X_train = data[..., 1:].permute(0, 3, 1, 2)
mask = (Y_train != 0)
Y_train = (Y_train // 5) - 1  # Przekształcenie do zakresu 0-17
Y_train[~mask] = -100  # Maskowanie klasy 0

dataset = TensorDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Model, loss i optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PixelClassifierCNN(num_features=6, num_classes=num_classes)

# Debugging: Print model summary
print(model)

model = model.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=-100)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trening
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.float().reshape(-1, num_classes)
        labels = labels.long().view(-1)
        # Debugging: Check the range of labels
        if torch.any(labels >= num_classes) or torch.any(labels < -1):
            print(f"Invalid label found: {labels}")
            continue
        try:
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        except RuntimeError as e:
            print(f"RuntimeError: {e}")
            print(f"Labels: {labels}")
            print(f"Outputs: {outputs}")
            raise e
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(dataloader):.4f}")

PixelClassifierCNN(
  (conv1): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc): Conv2d(128, 18, kernel_size=(1, 1), stride=(1, 1))
)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [2]:
import geopandas as gpd
noise = gpd.read_file('../data/NOISE.geojson')

In [3]:
noise

,DB_LO,DB_HI,SHAPE_LEN,geometry
0,5.0,10.0,7.617187e+05,"MULTIPOLYGON (((-733950.04 -1034130, -733950 -..."
1,10.0,15.0,8.163733e+05,"MULTIPOLYGON (((-733950.905 -1034130, -733950 ..."
2,15.0,20.0,8.648527e+05,"MULTIPOLYGON (((-733951.77 -1034130, -733950 -..."
3,20.0,25.0,9.233291e+05,"MULTIPOLYGON (((-733952.635 -1034130, -733950 ..."
4,25.0,30.0,1.016043e+06,"MULTIPOLYGON (((-733953.501 -1034130, -733950 ..."
5,30.0,35.0,1.280995e+06,"MULTIPOLYGON (((-733954.366 -1034130, -733950 ..."
6,35.0,40.0,2.186663e+06,"MULTIPOLYGON (((-733955.231 -1034130, -733950 ..."
7,40.0,45.0,3.608083e+06,"MULTIPOLYGON (((-733956.097 -1034130, -733950 ..."
8,45.0,50.0,4.391242e+06,"MULTIPOLYGON (((-733956.962 -1034130, -733950 ..."
9,50.0,55.0,4.295783e+06,"MULTIPOLYGON (((-733957.827 -1034130, -733950 ..."
